In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [54]:
# Load the dataset
df = pd.read_csv('uber.csv')

# Display the first few rows
df.head()
# Print df columns with types
print(df.dtypes)

Unnamed: 0             int64
key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object


### Handle missing values

In [55]:
# Checking for missing values
missing_values = df.isnull().sum()

# Drop rows with any missing values
df = df.dropna()

# Alternatively, fill missing values (if appropriate)
# df = df.fillna(method='ffill')

missing_values_after = df.isnull().sum()
missing_values_after

Unnamed: 0           0
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [56]:
# Validating latitude and longitude ranges
valid_longitude = (-180, 180)
valid_latitude = (-90, 90)

df = df[(df['pickup_longitude'].between(valid_longitude[0], valid_longitude[1])) &
        (df['pickup_latitude'].between(valid_latitude[0], valid_latitude[1])) &
        (df['dropoff_longitude'].between(valid_longitude[0], valid_longitude[1])) &
        (df['dropoff_latitude'].between(valid_latitude[0], valid_latitude[1]))]

# Filtering out rows with invalid fare_amount and passenger_count
df = df[df['fare_amount'] > 0]
df = df[df['passenger_count'] > 0]
df = df[df['passenger_count'] <= 6]

df.shape

(199256, 9)

## Data Transformation

In [57]:
# Extracting significant datetime information
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extracting features from pickup_datetime
df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_month'] = df['pickup_datetime'].dt.month
df['pickup_day'] = df['pickup_datetime'].dt.day
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_minute'] = df['pickup_datetime'].dt.minute
df['pickup_day_of_week'] = df['pickup_datetime'].dt.dayofweek
# 0 - Night , 1 - Morning, 2 - Afternoon, 3 - Evening
df['pickup_time_of_day'] = pd.cut(df['pickup_datetime'].dt.hour, bins=[0, 4, 12, 18, 24], labels=[0, 1, 2, 3], right=False)

# Create the naive datetime column (if not already created)
df['pickup_datetime_naive'] = df['pickup_datetime'].dt.tz_convert(None)

# Normalize to remove the time component
df['pickup_datetime_naive'] = df['pickup_datetime_naive'].dt.normalize()


# Display the first few rows after transformation
print("First few rows after datetime transformation:")
print(df.head())

df.to_csv('updated.csv', index=False)

First few rows after datetime transformation:
   Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

            pickup_datetime  pickup_longitude  pickup_latitude  \
0 2015-05-07 19:52:06+00:00        -73.999817        40.738354   
1 2009-07-17 20:04:56+00:00        -73.994355        40.728225   
2 2009-08-24 21:45:00+00:00        -74.005043        40.740770   
3 2009-06-26 08:22:21+00:00        -73.976124        40.790844   
4 2014-08-28 17:47:00+00:00        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  pickup_year  \
0         -73.999512         40.723217                1         2015   
1         -73.994710         40.750325        

## Data Extending
### Adding weather data to our dataset from the [link](https://www.ncei.noaa.gov/cdo-web/):

In [58]:
pre_processed_weather_df = pd.read_csv('nyc_weather.csv')
# Checking for missing values
missing_values = pre_processed_weather_df.isnull().sum()

missing_values

STATION       0
DATE          0
AWND         35
EVAP       2922
RHAV         80
RHMN         78
RHMX         78
SNOW          0
SNWD          0
TAVG       2922
TMAX          0
TMIN          0
TOBS       2922
TSUN       2922
WDMV       2922
WESD       2922
WSFM       2922
WT01       2040
WT03       2922
WT07       2871
WT14       2921
WT16       2344
WT17       2914
dtype: int64

In [59]:
weather_df = pre_processed_weather_df[['DATE', 'SNOW', 'SNWD', 'TMAX', 'TMIN']].copy() # Selecting only the required columns

weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])

weather_df.set_index('DATE', inplace=True)

weather_df.head()

,SNOW,SNWD,TMAX,TMIN
DATE,,,,
2009-01-01,0,0,-3.3,-9.4
2009-01-02,0,0,1.1,-5.0
2009-01-03,0,0,3.3,-1.7
2009-01-04,0,0,5.6,-3.9
2009-01-05,0,0,6.1,3.3


In [60]:
df['SNOW'] = df['pickup_datetime_naive'].map(weather_df['SNOW'])
df['SNWD'] = df['pickup_datetime_naive'].map(weather_df['SNWD'])
df['TMAX'] = df['pickup_datetime_naive'].map(weather_df['TMAX'])
df['TMIN'] = df['pickup_datetime_naive'].map(weather_df['TMIN'])

df.head()
df.to_csv('updated.csv', index=False)

### Holiday Data [Link](https://www.kaggle.com/datasets/donnetew/us-holiday-dates-2004-2021):


In [61]:
# Read the CSV file into a DataFrame
us_holidays_df = pd.read_csv('us_holidays.csv')

us_holidays_df['Date'] = pd.to_datetime(us_holidays_df['Date'])

# Display the DataFrame
us_holidays_df.dtypes

Date       datetime64[ns]
Holiday            object
WeekDay            object
Month               int64
Day                 int64
Year                int64
dtype: object

In [62]:
holidays_dates_df = us_holidays_df['Date'].copy()
holidays_dates_df.head()

0   2004-07-04
1   2005-07-04
2   2006-07-04
3   2007-07-04
4   2008-07-04
Name: Date, dtype: datetime64[ns]

In [63]:
df = df.merge(holidays_dates_df, left_on='pickup_datetime_naive', right_on='Date', how='left')
df['is_holiday'] = df['Date'].notnull().astype(int)
df.drop(columns='Date', inplace=True)
df.head()
df.sort_values(by='pickup_datetime', inplace=True)
df.to_csv('uber_updated.csv', index=False)

In [64]:
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,...,pickup_hour,pickup_minute,pickup_day_of_week,pickup_time_of_day,pickup_datetime_naive,SNOW,SNWD,TMAX,TMIN,is_holiday
100584,55072468,2009-01-01 01:15:22.0000006,8.5,2009-01-01 01:15:22+00:00,-73.981918,40.779456,-73.957685,40.771043,2,2009,...,1,15,3,0,2009-01-01,0,0,-3.3,-9.4,1
43861,30813112,2009-01-01 01:59:17.0000001,13.0,2009-01-01 01:59:17+00:00,-73.983759,40.721389,-73.994833,40.687179,2,2009,...,1,59,3,0,2009-01-01,0,0,-3.3,-9.4,1
7616,8688426,2009-01-01 02:05:03.0000003,10.6,2009-01-01 02:05:03+00:00,-73.956635,40.771254,-73.991528,40.749778,2,2009,...,2,5,3,0,2009-01-01,0,0,-3.3,-9.4,1
118451,13190369,2009-01-01 02:09:13.0000003,12.2,2009-01-01 02:09:13+00:00,-73.984605,40.728020,-73.955746,40.776830,1,2009,...,2,9,3,0,2009-01-01,0,0,-3.3,-9.4,1
89530,45716268,2009-01-01 02:13:41.0000001,11.0,2009-01-01 02:13:41+00:00,-73.980127,40.737425,-74.009544,40.726025,4,2009,...,2,13,3,0,2009-01-01,0,0,-3.3,-9.4,1


# adding distance (km) of drive

In [65]:
from haversine import haversine

def calculate_distance(row):
    pickup = (row['pickup_latitude'], row['pickup_longitude'])
    dropoff = (row['dropoff_latitude'], row['dropoff_longitude'])
    return haversine(pickup, dropoff)

df['distance'] = df.apply(calculate_distance, axis=1)
df.sort_values(by='pickup_datetime', inplace=True)
df.to_csv('uber_updated.csv', index=False)

In [66]:
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,...,pickup_minute,pickup_day_of_week,pickup_time_of_day,pickup_datetime_naive,SNOW,SNWD,TMAX,TMIN,is_holiday,distance
100584,55072468,2009-01-01 01:15:22.0000006,8.5,2009-01-01 01:15:22+00:00,-73.981918,40.779456,-73.957685,40.771043,2,2009,...,15,3,0,2009-01-01,0,0,-3.3,-9.4,1,2.244768
43861,30813112,2009-01-01 01:59:17.0000001,13.0,2009-01-01 01:59:17+00:00,-73.983759,40.721389,-73.994833,40.687179,2,2009,...,59,3,0,2009-01-01,0,0,-3.3,-9.4,1,3.916847
7616,8688426,2009-01-01 02:05:03.0000003,10.6,2009-01-01 02:05:03+00:00,-73.956635,40.771254,-73.991528,40.749778,2,2009,...,5,3,0,2009-01-01,0,0,-3.3,-9.4,1,3.786742
118451,13190369,2009-01-01 02:09:13.0000003,12.2,2009-01-01 02:09:13+00:00,-73.984605,40.728020,-73.955746,40.776830,1,2009,...,9,3,0,2009-01-01,0,0,-3.3,-9.4,1,5.946965
89530,45716268,2009-01-01 02:13:41.0000001,11.0,2009-01-01 02:13:41+00:00,-73.980127,40.737425,-74.009544,40.726025,4,2009,...,13,3,0,2009-01-01,0,0,-3.3,-9.4,1,2.784026


# dividing by the treatment

In [67]:
early_df=df[(df["pickup_hour"]==7)|(df["pickup_hour"]==8)][(df["pickup_day_of_week"]!=6)&(df["pickup_day_of_week"]!=5)]
early_df

C:\Users\Inbar\AppData\Local\Temp\ipykernel_2804\2370099949.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  early_df=df[(df["pickup_hour"]==7)|(df["pickup_hour"]==8)][(df["pickup_day_of_week"]!=6)&(df["pickup_day_of_week"]!=5)]


,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,...,pickup_minute,pickup_day_of_week,pickup_time_of_day,pickup_datetime_naive,SNOW,SNWD,TMAX,TMIN,is_holiday,distance
182970,30562954,2009-01-02 07:21:00.0000007,9.3,2009-01-02 07:21:00+00:00,-73.983085,40.729027,-74.006095,40.707702,1,2009,...,21,4,1,2009-01-02,0,0,1.1,-5.0,0,3.063226
35969,28442656,2009-01-02 07:21:39.0000001,10.1,2009-01-02 07:21:39+00:00,-73.961482,40.764459,-73.991651,40.749690,1,2009,...,21,4,1,2009-01-02,0,0,1.1,-5.0,0,3.025573
28775,26188451,2009-01-02 07:42:00.00000049,6.1,2009-01-02 07:42:00+00:00,-73.981670,40.766260,-73.996820,40.744107,1,2009,...,42,4,1,2009-01-02,0,0,1.1,-5.0,0,2.774220
154975,50129115,2009-01-02 08:10:00.00000010,16.1,2009-01-02 08:10:00+00:00,0.000000,0.000000,0.000000,0.000000,1,2009,...,10,4,1,2009-01-02,0,0,1.1,-5.0,0,0.000000
46968,50607326,2009-01-05 07:06:00.00000097,4.1,2009-01-05 07:06:00+00:00,-74.006237,40.716960,-74.010613,40.709268,1,2009,...,6,0,1,2009-01-05,0,0,6.1,3.3,0,0.931447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82645,48730246,2015-06-30 07:46:24.0000001,17.5,2015-06-30 07:46:24+00:00,-73.977356,40.787106,-74.008736,40.724033,1,2015,...,46,1,1,2015-06-30,0,0,27.8,20.0,0,7.494849
131689,51123054,2015-06-30 07:52:34.0000002,6.0,2015-06-30 07:52:34+00:00,-73.984367,40.764740,-73.987572,40.753059,2,2015,...,52,1,1,2015-06-30,0,0,27.8,20.0,0,1.326570
43331,21446442,2015-06-30 08:16:19.0000003,6.5,2015-06-30 08:16:19+00:00,-73.975037,40.755051,-73.974899,40.763435,1,2015,...,16,1,1,2015-06-30,0,0,27.8,20.0,0,0.932410
136198,43329667,2015-06-30 08:40:33.0000003,10.5,2015-06-30 08:40:33+00:00,-73.983665,40.725780,-73.995193,40.740746,4,2015,...,40,1,1,2015-06-30,0,0,27.8,20.0,0,1.926791


In [68]:
late_df=df[(df["pickup_hour"]==9)|(df["pickup_hour"]==10)][(df["pickup_day_of_week"]!=6)&(df["pickup_day_of_week"]!=5)]
late_df

C:\Users\Inbar\AppData\Local\Temp\ipykernel_2804\4133159960.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  late_df=df[(df["pickup_hour"]==9)|(df["pickup_hour"]==10)][(df["pickup_day_of_week"]!=6)&(df["pickup_day_of_week"]!=5)]


,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,...,pickup_minute,pickup_day_of_week,pickup_time_of_day,pickup_datetime_naive,SNOW,SNWD,TMAX,TMIN,is_holiday,distance
119927,50042881,2009-01-01 10:11:29.0000003,27.45,2009-01-01 10:11:29+00:00,-73.982014,40.777893,-73.885535,40.773202,2,2009,...,11,3,1,2009-01-01,0,0,-3.3,-9.4,1,8.140754
89651,45824144,2009-01-01 10:35:00.00000076,10.10,2009-01-01 10:35:00+00:00,-73.972288,40.761978,-73.984402,40.729590,2,2009,...,35,3,1,2009-01-01,0,0,-3.3,-9.4,1,3.743186
135456,30127562,2009-01-02 09:11:19.0000003,26.10,2009-01-02 09:11:19+00:00,-73.978024,40.765521,-73.872502,40.774423,1,2009,...,11,4,1,2009-01-02,0,0,1.1,-5.0,0,8.941200
32419,35105832,2009-01-02 09:15:00.0000005,4.90,2009-01-02 09:15:00+00:00,-73.972077,40.794502,-73.978340,40.783228,1,2009,...,15,4,1,2009-01-02,0,0,1.1,-5.0,0,1.359986
43618,37579593,2009-01-02 10:52:00.000000113,14.10,2009-01-02 10:52:00+00:00,-73.992347,40.759503,-73.997868,40.716843,1,2009,...,52,4,1,2009-01-02,0,0,1.1,-5.0,0,4.766334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74808,47500388,2015-06-29 10:54:24.0000001,42.04,2015-06-29 10:54:24+00:00,-73.898399,40.753639,-73.980766,40.764999,1,2015,...,54,0,1,2015-06-29,0,0,24.4,17.2,0,7.051479
22690,19806831,2015-06-30 09:20:08.0000005,4.00,2015-06-30 09:20:08+00:00,-73.992592,40.742905,-73.986702,40.739891,1,2015,...,20,1,1,2015-06-30,0,0,27.8,20.0,0,0.598765
42042,15896882,2015-06-30 09:55:02.0000003,29.04,2015-06-30 09:55:02+00:00,-73.863350,40.769672,-73.944633,40.808380,1,2015,...,55,1,1,2015-06-30,0,0,27.8,20.0,0,8.084147
6887,50164520,2015-06-30 09:55:45.0000003,20.00,2015-06-30 09:55:45+00:00,-73.993446,40.729092,-73.982979,40.766621,1,2015,...,55,1,1,2015-06-30,0,0,27.8,20.0,0,4.265185
